<a href="https://colab.research.google.com/github/SwathiNagilla/Swathi_INFO5731_FALL2024/blob/main/INFO5731_Assignment_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#Function to Extract movie reviews from IMDB
def get_reviews(movie_url, max_reviews=1000):
    reviews = []
    page = 1

    while len(reviews) < max_reviews:
      #set header
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
        }

        url = f"{movie_url}reviews/_ajax?ref_=undefined&paginationKey=page={page}"
        #Make a GET request to fetch the reviews
        response = requests.get(url, headers=headers)
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f'Failed to retrieve reviews for page {page}. Status code: {response.status_code}')
            break
        #Parse the response content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract reviews
        review_elements = soup.find_all('div', class_='review-container')
        if not review_elements:
            print(" ")
            break

        for review_element in review_elements:
            try:
              #Extract the text of the review
                review_text = review_element.find('div', class_='text').get_text(strip=True)
                reviews.append(review_text)
                if len(reviews) >= max_reviews:
                    break
            except AttributeError:
                continue

        print(f'Collected {len(reviews)} reviews from page {page}.')
        page += 1
    #Return the list of collected reviews
    return reviews
#Function to save reviews to CSV file
def save_reviews_to_csv(reviews, filename='movie_reviews.csv'):
    df = pd.DataFrame(reviews, columns=['Review'])
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f'Saved {len(reviews)} reviews to {filename}.')

# IMDb movie URL for reviews
movie_url = 'https://www.imdb.com/title/tt1431045/reviews/#'
all_reviews = get_reviews(movie_url)
save_reviews_to_csv(all_reviews)


Collected 25 reviews from page 1.
Collected 50 reviews from page 2.
Collected 75 reviews from page 3.
Collected 100 reviews from page 4.
Collected 125 reviews from page 5.
Collected 150 reviews from page 6.
Collected 175 reviews from page 7.
Collected 200 reviews from page 8.
Collected 225 reviews from page 9.
Collected 250 reviews from page 10.
Collected 275 reviews from page 11.
 
Saved 275 reviews to movie_reviews.csv.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [47]:
# Importing necessary libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the reviews
data = pd.read_csv('movie_reviews.csv')

def clean(text):
    # 1. Remove noise, special characters, and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # 2. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3. Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)

    # 4. Lowercase all texts
    text = text.lower()

    # 5. Stemming
    stemmer = PorterStemmer()
    text = ' '.join(stemmer.stem(word) for word in text.split())

    # 6. Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())

    return text

# Clean the reviews and save to a new column
data['Cleaned_Review'] = data['Review'].apply(clean)

# Save the cleaned data to a new CSV file
data.to_csv('cleaned_movie_reviews.csv', index=False, encoding='utf-8')
print('Saved cleaned reviews to cleaned_movie_reviews.csv.')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Saved cleaned reviews to cleaned_movie_reviews.csv.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [17]:
#necessary libraries
!pip install nltk spacy
!python -m spacy download en_core_web_sm
import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize, ne_chunk, RegexpParser
from nltk.tree import Tree
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [48]:

# Read the file
data= pd.read_csv('cleaned_movie_reviews.csv')

# Function to perform POS tagging and count occurrences
def pos(text):
    token = word_tokenize(text)
    pos_tags = pos_tag(token)
    return pos_tags

# Seperate POS tags for all reviews
all_tags = []
for review in df['Cleaned_Review']:
    all_tags.extend(pos(review))

# Count occurrences of each POS tag
pos_counts = Counter(tag for word, tag in all_tags)
nouns = pos_counts['NN'] + pos_counts['NNS'] + pos_counts['NNP'] + pos_counts['NNPS']
verbs = pos_counts['VB'] + pos_counts['VBD'] + pos_counts['VBG'] + pos_counts['VBN'] + pos_counts['VBP'] + pos_counts['VBZ']
adjectives = pos_counts['JJ'] + pos_counts['JJR'] + pos_counts['JJS']
adverbs = pos_counts['RB'] + pos_counts['RBR'] + pos_counts['RBS']

print(f'Total Nouns: {nouns}')
print(f'Total Verbs: {verbs}')
print(f'Total Adjectives: {adjectives}')
print(f'Total Adverbs: {adverbs}')

# Function for constituency parsing
def constituency_parsing(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    grammar = r"""
      NP: {<DT>?<JJ>*<NN.*>}   # Noun Phrase
      VP: {<VB.*><NP|PP>*}      # Verb Phrase
      """
    cp = RegexpParser(grammar)
    tree = cp.parse(pos_tags)
    return tree

# Take review from the dataset
sentence = data['Cleaned_Review'].iloc[0]
constituency_tree = constituency_parsing(sentence)
print("\nConstituency Parsing Tree:")
print(constituency_tree)

# Function for dependency parsing using spaCy
import spacy
nlp = spacy.load("en_core_web_sm")

def dependency_parsing(text):
    doc = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in doc]

dependency_tree = dependency_parsing(sentence)
print("\nDependency Parsing Tree:")
for token in dependency_tree:
    print(token)

# Function for Named Entity Recognition
def named_entity_recognition(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Aggregate NER results
all_entities = []
for review in data['Cleaned_Review']:
    all_entities.extend(named_entity_recognition(review))

# Count entities
entity_counts = Counter(entity for entity, label in all_entities)
print("\nNamed Entity Counts:")
for entity, count in entity_counts.items():
    print(f"{entity}: {count}")


Total Nouns: 17616
Total Verbs: 3840
Total Adjectives: 6192
Total Adverbs: 1408

Constituency Parsing Tree:
(S
  (NP deadpool/NN)
  (NP absolut/NN)
  (NP hilari/NN)
  (NP stori/NN)
  (NP isnt/JJ groundbreak/NN)
  (NP trick/NN)
  (NP goe/NN)
  (NP action/NN)
  (NP isnt/NN)
  bad/JJ
  either/RB
  (NP great/JJ film/NN)
  (NP lie/NN)
  (NP humor/NN)
  (NP uniqu/JJ isnt/NN)
  like/IN
  (NP everi/NN)
  (NP superhero/NN)
  (NP film/NN)
  whether/IN
  (NP talk/NN)
  (NP constant/JJ break/NN)
  (NP fourth/JJ wall/NN)
  (NP rrate/NN)
  (VP
    differ/VBP
    (NP characterist/NN)
    (NP film/NN)
    (NP differenti/NN)
    (NP superhero/NN)
    (NP movi/NN))
  (VP frank/VBD (NP arguabl/JJ doesnt/NN))
  even/RB
  (VP
    justifi/VBZ
    (NP superhero/NN)
    (NP stamp/NN)
    (NP someth/NN)
    (NP differentit/NN)
    (NP need/NN))
  (VP said/VBD (NP film/NN))
  might/MD
  (VP
    everyon/VB
    (NP crude/NN)
    (NP fill/NN)
    (NP nuditi/JJ violenc/NN)
    (NP humor/NN)
    (NP wont/JJ appeal/N

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

https://myunt-my.sharepoint.com/:x:/g/personal/swathinagilla_my_unt_edu/Eb91etIja8VMnXWOwuWhNTQBSDMxyqnMB_gym2tZE_kNHg?e=saTiKg

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [6]:
'''The assignment was a good learning experience for web scraping and data analysis. I struggled a bit with pulling reviews from IMDb
because the website's structure kept changing and I found it hard to manage my time on
all the tasks. Still, I enjoyed learning about scraping, seeing my code work, and figuring out how to clean the data. The time given
felt tight but it pushed me to work efficiently.'''